# Intro to Using the NetHack Learning Dataset

There are two different sets of trajectories included in the NetHack Learning Dataset:
- **NLD-NAO**: state-only trajectories from 1.5 M human games played on nethack.alt.org
- **NLD-AA**: state-action-score trajectories from 100k NLE games played by the symbolic-bot winner of the 2021 NetHack Challenge

We also supply a small "taster" dataset, for quick iteration and playing around:
- **NLD-AA-Taster**: ~2,000 randomly chosen trajectories from **NLD-AA**

These trajectories can be used with the `TtyrecDataset` tool which allows for efficiently training on the datasets.  This tutorial describes how to create and use and visualize the dataset, using **NLD-AA-Taster**.



## Downloading the Data

For the time being data is available through various WeTransfer links in the DATASET.md file. Although generally this requires a browser to interface to download, it is also possible to use the command line (see here).

In this case, we use a publically available unzipped version of **NLD-AA-Taster** available on GitHub (or you can access the [zipped release]()).


## Install NLE

Make sure you have `nle` installed by following the instructions [in the repo README here](https://github.com/facebookresearch/nle). Either clone and install, or use pip. In this case, Colab struggles a bit to find cmake so we build from source:

## Setting up the Database

Adding datasets is easy - all you need is the path to the unzipped directory.

**NOTE** We call different functions to add trajectories generated by NLE (such as **NLD-AA**, **NLD-AA-Taster** or your own dataset) versus those generated from NAO (**NLD-NAO**).  

In [5]:
import nle.dataset as nld

In [6]:
# 1. Get the paths for your unzipped datasets
path_to_nld_aa_taster = "./data/nld-aa-taster/nle_data"

# 2. Chose a database name/path. By default, most methods with use nld.db.DB (='ttyrecs.db')
dbfilename = "ttyrecs.db"

if not nld.db.exists(dbfilename):
    # 3. Create the db and add the directory
    nld.db.create(dbfilename)
    nld.add_nledata_directory(path_to_nld_aa_taster, "taster-dataset", dbfilename)


# NB: To add the NLE-AA data, or any data generated from nle, use `add_nledata_directory`.
# nld.add_nledata_directory(path_to_nld_aa, "nld-aa", dbfilename)

# NB: To add the NLE-NAO data, use the `add_altorg_directory`.
# nld.add_altorg_directory(path_to_nld_nao, "nld-nao", dbfilename)


In [47]:
path_to_nld_aa_training = "./data/nld-aa/nle_data_train"
path_to_nld_aa_testing = "./data/nld-aa/nle_data_test"
nld.add_nledata_directory(path_to_nld_aa_training, "nld-aa-training", dbfilename)
nld.add_nledata_directory(path_to_nld_aa_testing, "nld-aa-testing", dbfilename)

Adding dataset 'nld-aa-training' ('./data/nld-aa/nle_data_train') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.70 sec. Size: 4.12 MB, Games: 11194
Adding dataset 'nld-aa-testing' ('./data/nld-aa/nle_data_test') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.70 sec. Size: 4.12 MB, Games: 11194
Adding dataset 'nld-aa-testing' ('./data/nld-aa/nle_data_test') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.17 sec. Size: 4.96 MB, Games: 2709
Updated 'ttyrecs.db' in 0.17 sec. Size: 4.96 MB, Games: 2709


In [49]:
path_to_nld_nao_training = "./data/nld-nao/nld_nao_train"
path_to_nld_nao_testing = "./data/nld-nao/nld_nao_test"
nld.add_altorg_directory(path_to_nld_nao_training, "nld-nao-training", dbfilename)
nld.add_altorg_directory(path_to_nld_nao_testing, "nld-nao-testing", dbfilename)

Adding dataset 'nld-nao-training' ('./data/nld-nao/nld_nao_train') to 'ttyrecs.db' 
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+:Zone.Identifier'
Found 1736841 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh362:Zone.Identifier'
Found 1736841 games in './data/nld-nao/nld_nao_train/xlogfile.nh363+'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh362:Zone.Identifier'
Found 167705 games in './data/nld-nao/nld_nao_train/xlogfile.nh362'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev:Zone.Identifier'
Found 167705 games in './data/nld-nao/nld_nao_train/xlogfile.nh362'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev:Zone.Identifier'
Found 20939 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev'
Found 0 games in './data/nld-nao/nld_nao_train/xlogfile.nh361:Zone.Identifier'
Found 20939 games in './data/nld-nao/nld_nao_train/xlogfile.nh361dev'
Found 0 games in '.

You can inspect the dataset using the database tooling:

In [7]:
# Create a connection to specify the database to use
db_conn = nld.db.connect(filename=dbfilename)

# Then you can inspect the number of games in each dataset:
print(f"NLD AA \"Taster\" Dataset has {nld.db.count_games('taster-dataset', conn=db_conn)} games.")

NLD AA "Taster" Dataset has 1934 games.


## Visualizing the Data

Next, to actually load the games for training you'll use the `TtyrecDataset` object:

In [8]:
dataset = nld.TtyrecDataset(
    "nld-aa-training",
    batch_size=4,
    seq_length=32,
    dbfilename=dbfilename
)

This dataset above will return batches of 128 trajectories, returning sequential chunks of length 32.   That is, assuming the length of all trajectories is >>64, the first batch will give timesteps 0-31 of 128 games and the second batch will provide timesteps 32-63 for the same games, etc.

### Whats in the Observation?

In [9]:
minibatch = next(iter(dataset))
minibatch.keys()

dict_keys(['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores'])

In [10]:
minibatch['keypresses'][0]

array([ 32,  27,  27,  24,  32,  32, 229,  32,  32,  64,  92,  32,  97,
       104,  58,  32,  32,  67, 105, 104,  35,  48,  13,  97, 104,  58,
        32,  32,  58,  47,  77,  32], dtype=uint8)

In [86]:
''.join([chr(a) for a in minibatch['tty_chars'][0][0][3]])

'                                                     -----                      '

The observation is made up of three components:
- `tty_chars` is a (batched) 2D np.array of the characters displayed at each point on the screen with shape: `[Batch, Time, H, W]`
- `tty_colors` is the associated colors for those characters
- `tty_cursor` provides the cursor position (NOTE: it's not always on the hero!)

These can be easily visualized usign the `tty_render` utility:

In [19]:
from nle.nethack import tty_render

In [98]:
batch_idx = 0
time_idx = 0
chars = minibatch['tty_chars'][batch_idx, time_idx]
colors = minibatch['tty_colors'][batch_idx, time_idx]
cursor = minibatch['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))


Hello Agent, welcome to NetHack!  You are a neutral female gnomish              
                                                                  Archeologist.-
-More--                                                                         
                                                     -----                      
                                                     +...+                      
                                                     |[({|                      
                                                     .f..|                      
                                                     |@..-                      
                                                     +...|                      
                                                     -----                      
                                                                                
                                                                                
                           

### Extracting Inventory Information from Dataset

The dataset contains several ways to access inventory information:
1. **From TTY rendering** - parsing the visual inventory display
2. **From minibatch keys** - if available in the dataset
3. **By triggering inventory commands** - looking for 'i' keypresses

Let's explore all these methods:

In [39]:
# First, let's see what keys are available in our minibatch
print("=== Available Data in Minibatch ===")
print("Keys:", list(minibatch.keys()))
print()

# Check if inventory-related keys exist
inventory_keys = ['inv_glyphs', 'inv_letters', 'inv_oclasses', 'inv_strs']
available_inv_keys = [key for key in inventory_keys if key in minibatch.keys()]
print(f"Inventory-specific keys available: {available_inv_keys}")

# If no direct inventory keys, we'll need to parse from TTY or find inventory screens
if not available_inv_keys:
    print("No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.")
else:
    print("Great! We have direct inventory data available.")

print(f"\nMinibatch shapes:")
for key, value in minibatch.items():
    if hasattr(value, 'shape'):
        print(f"  {key}: {value.shape}")
    else:
        print(f"  {key}: {type(value)}")

=== Available Data in Minibatch ===
Keys: ['tty_chars', 'tty_colors', 'tty_cursor', 'timestamps', 'done', 'gameids', 'keypresses', 'scores']

Inventory-specific keys available: []
No direct inventory keys found. We'll need to extract from TTY rendering or find inventory commands.

Minibatch shapes:
  tty_chars: (32, 32, 24, 80)
  tty_colors: (32, 32, 24, 80)
  tty_cursor: (32, 32, 2)
  timestamps: (32, 32)
  done: (32, 32)
  gameids: (32, 32)
  keypresses: (32, 32)
  scores: (32, 32)


In [40]:
# Method 1: Find when players opened inventory (pressed 'i')
print("=== Method 1: Finding Inventory Commands ===")

# Look for inventory keypresses (ASCII 105 = 'i')
inventory_keypress = ord('i')  # 105

# Search through the dataset for inventory commands
batch_with_inventory = None
timestep_with_inventory = None

for batch_idx in range(min(5, minibatch['keypresses'].shape[0])):  # Check first 5 batches
    for time_idx in range(minibatch['keypresses'].shape[1]):
        if minibatch['keypresses'][batch_idx, time_idx] == inventory_keypress:
            batch_with_inventory = batch_idx
            timestep_with_inventory = time_idx
            print(f"Found inventory command at batch {batch_idx}, timestep {time_idx}")
            break
    if batch_with_inventory is not None:
        break

if batch_with_inventory is None:
    print("No inventory commands found in current minibatch. Let's create a larger search...")
    
    # Create a larger dataset to search for inventory
    larger_dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=10,
        seq_length=100,
        dbfilename=dbfilename,
    )
    
    # Search multiple batches
    found_inventory = False
    for mb_idx, large_mb in enumerate(larger_dataset):
        if mb_idx > 3:  # Don't search forever
            break
            
        inventory_positions = (large_mb['keypresses'] == inventory_keypress)
        if inventory_positions.any():
            # Find the first occurrence
            batch_indices, time_indices = inventory_positions.nonzero()
            batch_with_inventory = batch_indices[0]
            timestep_with_inventory = time_indices[0]
            
            print(f"Found inventory command in batch {mb_idx}, game {batch_with_inventory}, timestep {timestep_with_inventory}")
            
            # Use this minibatch for analysis
            minibatch_with_inv = large_mb
            found_inventory = True
            break
    
    if not found_inventory:
        print("No inventory commands found. Using current minibatch for demonstration.")
        minibatch_with_inv = minibatch
        batch_with_inventory = 0
        timestep_with_inventory = 10  # Just pick a timestep
else:
    minibatch_with_inv = minibatch

print(f"Using batch {batch_with_inventory}, timestep {timestep_with_inventory} for analysis")

=== Method 1: Finding Inventory Commands ===
Found inventory command at batch 1, timestep 17
Using batch 1, timestep 17 for analysis


In [41]:
# Method 2: Analyze the inventory screen
print("=== Method 2: Analyzing Inventory Screen ===")

# Look at the screen right after the inventory command
if timestep_with_inventory + 1 < minibatch_with_inv['tty_chars'].shape[1]:
    next_timestep = timestep_with_inventory + 1
else:
    next_timestep = timestep_with_inventory

chars_before = minibatch_with_inv['tty_chars'][batch_with_inventory, timestep_with_inventory]
colors_before = minibatch_with_inv['tty_colors'][batch_with_inventory, timestep_with_inventory]
cursor_before = minibatch_with_inv['tty_cursor'][batch_with_inventory, timestep_with_inventory]

chars_after = minibatch_with_inv['tty_chars'][batch_with_inventory, next_timestep]
colors_after = minibatch_with_inv['tty_colors'][batch_with_inventory, next_timestep]
cursor_after = minibatch_with_inv['tty_cursor'][batch_with_inventory, next_timestep]

print("Screen BEFORE inventory command:")
print(tty_render(chars_before, colors_before, cursor_before))
print("\n" + "="*80 + "\n")

print("Screen AFTER inventory command:")
print(tty_render(chars_after, colors_after, cursor_after))

# Method 3: Extract inventory information from the text
print("\n=== Method 3: Parsing Inventory Information ===")

def extract_inventory_from_screen(chars):
    """Extract inventory items from TTY characters"""
    inventory_items = []
    
    # Convert chars to string representation
    screen_lines = []
    for row in range(chars.shape[0]):
        line = ''.join([chr(c) if 32 <= c <= 126 else ' ' for c in chars[row]])
        screen_lines.append(line.rstrip())
    
    # Look for common inventory patterns
    for i, line in enumerate(screen_lines):
        line = line.strip()
        
        # NetHack inventory lines typically start with a letter followed by ')'
        if len(line) > 2 and line[1] == ')' and line[0].isalpha():
            inventory_items.append({
                'slot': line[0],
                'description': line[2:].strip(),
                'line_number': i
            })
        
        # Also look for "You are carrying:" or similar inventory headers
        if 'carrying' in line.lower() or 'inventory' in line.lower():
            print(f"Inventory header found at line {i}: '{line}'")
    
    return inventory_items, screen_lines

# Extract from the inventory screen
inv_items, screen_lines = extract_inventory_from_screen(chars_after)

print(f"Found {len(inv_items)} inventory items:")
for item in inv_items:
    print(f"  {item['slot']}) {item['description']}")

if len(inv_items) == 0:
    print("No inventory items found in standard format.")
    print("Screen might show a different interface. Here are all non-empty lines:")
    for i, line in enumerate(screen_lines):
        if line.strip():
            print(f"  Line {i:2d}: '{line}'")

=== Method 2: Analyzing Inventory Screen ===
Screen BEFORE inventory command:

                                What do you want to name?                       
                                                                                
                                m - a monster                                   
                                i - a particular object in inventory            
                                o - the type of an object in inventory          
                                f - the type of an object upon the floor        
                                d - the type of an object on discoveries list   
                                a - record an annotation for the current level  
                                (end)                                           
                                                                                
                                                                                
                              

Then, the other elements of the batch are:
- `gameids`: The gameid for the game which the observation is from.
- `timestamps`: The time when the state was recorded, allowing you to understand how long the player took between frames.
- `keypresses`: The keypresses entered after seeing the observation at this timestep (which produces the observation at the next timestep).
- `scores`: The in-game score at this timestep (the result of the action at the previous timestep)
- `done`: Whether the gameid corresponding to the previous timestep's observation completed. If done is `True` this means that the observation at the current timestep is the beginning of the next gameid.

### Converting Actions from Keypresses to Environment Action Space

Note that the "actions" data is actually a keypress (eg ascii) entered not an action value corresponding to the actions in the nle environment.  To convert from keypresses to the action_space of the environment you can use an embedding as shown below:

In [11]:
# Fix NetHack installation path
import os
import sys

# Set the correct path to nethackdir
nethack_path = "/home/xchen/UCL_thesis/SequentialSkillRL/nle/nle/nethackdir"
if os.path.exists(nethack_path):
    os.environ['HACKDIR'] = nethack_path
    print(f"Set HACKDIR to: {nethack_path}")
else:
    print(f"Warning: NetHack directory not found at {nethack_path}")
    print("Available directories:")
    base_path = "/home/xchen/UCL_thesis/SequentialSkillRL/nle"
    if os.path.exists(base_path):
        for item in os.listdir(base_path):
            item_path = os.path.join(base_path, item)
            if os.path.isdir(item_path):
                print(f"  {item_path}")
                # Check if this directory contains nethackdir
                nethack_subdir = os.path.join(item_path, "nethackdir")
                if os.path.exists(nethack_subdir):
                    print(f"    -> Found nethackdir at: {nethack_subdir}")
                    os.environ['HACKDIR'] = nethack_subdir

Set HACKDIR to: /home/xchen/UCL_thesis/SequentialSkillRL/nle/nle/nethackdir


In [12]:
import torch
from nle.nle.env.tasks import NetHackChallenge

# Method 1: Create symbolic link if it doesn't exist
expected_path = "/home/xchen/UCL_thesis/SequentialSkillRL/nle/nethackdir"
actual_path = "/home/xchen/UCL_thesis/SequentialSkillRL/nle/nle/nethackdir"

if not os.path.exists(expected_path) and os.path.exists(actual_path):
    try:
        os.symlink(actual_path, expected_path)
        print(f"✅ Created symbolic link: {expected_path} -> {actual_path}")
    except FileExistsError:
        print(f"Symbolic link already exists: {expected_path}")
    except Exception as e:
        print(f"Failed to create symbolic link: {e}")

# Method 2: Try creating environment with direct hackdir parameter
try:
    # Try with hackdir parameter first
    from nle.nle.env.base import NLE
    
    env = NetHackChallenge(
        savedir=None,  # Do not save any recordings. 
        character='@', # Randomly rotate through characters.
        hackdir=actual_path  # Direct path specification
    )
    print("✅ NetHackChallenge environment created successfully with hackdir parameter!")
    
except Exception as e1:
    print(f"❌ Failed with hackdir parameter: {e1}")
    
    # Fallback: Try with symbolic link
    try:
        env = NetHackChallenge(
            savedir=None,
            character='@',
        )
        print("✅ NetHackChallenge environment created successfully with symbolic link!")
        
    except Exception as e2:
        print(f"❌ Failed with symbolic link: {e2}")
        
        # Last resort: Try environment variables
        os.environ['NETHACKDIR'] = actual_path
        os.environ['HACKDIR'] = actual_path
        os.environ['NLE_HACKDIR'] = actual_path
        
        try:
            env = NetHackChallenge(
                savedir=None,
                character='@',
            )
            print("✅ NetHackChallenge environment created with environment variables!")
            
        except Exception as e3:
            print(f"❌ All methods failed. Final error: {e3}")
            print("\nDebugging information:")
            print(f"Expected path exists: {os.path.exists(expected_path)}")
            print(f"Actual path exists: {os.path.exists(actual_path)}")
            print(f"nhdat exists: {os.path.exists(os.path.join(actual_path, 'nhdat'))}")
            raise e3

# Then use the environment actions to convert the keypresses.
embed_actions = torch.zeros((256, 1))
for i, a in enumerate(env.actions):
    embed_actions[a.value][0] = i
    
embed_actions = torch.nn.Embedding.from_pretrained(embed_actions)
keypresses = torch.Tensor(minibatch["keypresses"]).long()
actions = embed_actions(keypresses).squeeze(-1).long()

print(f"✅ Action embedding created successfully!")
print(f"   Embedding shape: {embed_actions.weight.shape}")
print(f"   Number of unique actions: {len(env.actions)}")
print(f"   Actions tensor shape: {actions.shape}")

❌ Failed with hackdir parameter: NLE.__init__() got an unexpected keyword argument 'hackdir'
✅ NetHackChallenge environment created successfully with symbolic link!
✅ Action embedding created successfully!
   Embedding shape: torch.Size([256, 1])
   Number of unique actions: 121
   Actions tensor shape: torch.Size([4, 32])


In [25]:
actions

tensor([[ 38,  38,  25, 107, 107,  37, 107, 107,  26,  49, 107,  51, 101,  55,
         107,  20,  91,  35,  19,   6,  38,  38,  38,  20,  91,  35,  19,   6,
          38,  49, 107,  51],
        [107,  38,  38,  25, 107, 107,  37, 107, 107,  26,  49, 107,  24,   3,
          51, 107, 107,  27,  44,   3,  20, 110,  19,  24,   3,  51, 107, 107,
          51, 101,  55, 107],
        [ 38,  38,  25, 107, 107,  37, 107, 107,  26,  49, 107, 107, 107,  51,
         101,  55, 107,  20,  91,  35,  19,   6,  38,  38,  38,  20,  91,  35,
          19,   6,  38,  49],
        [ 38,  38,  25, 107, 107,  37, 107,  26,  49, 107,  51, 101,  55, 107,
          20,  91,  35,  19,   6,  38,  38,  38,  20,  91,  35,  19,   6,  38,
          49, 107,  51, 101]])

In [31]:
env.actions[0]

<CompassDirection.N: 107>

In [30]:
minibatch['keypresses'][0]

array([ 27,  27,  24,  32,  32, 229,  32,  32,  64,  92,  32,  58,  47,
        77,  32,  35, 116, 101,  13,  98,  27,  27,  27,  35, 116, 101,
        13,  98,  27,  92,  32,  58], dtype=uint8)

In [1]:
from utils.action_utils import keypress_to_action_index, action_abbr_name, action_full_name

In [13]:
action_idx = keypress_to_action_index(env, 107)
print(action_abbr_name(action_idx))

ValueError: Action 0 is not a valid Nethack action

In [17]:
from nle import nethack
len(nethack.ACTIONS)

121

In [18]:
len(env.actions)

121

In [19]:
nethack.ACTIONS == env.actions

True

In [27]:
from nle.nethack.actions import action_id_to_type
action_id_to_type(107).value

AttributeError: 'str' object has no attribute 'value'

In [24]:
values = [a.value for a in nethack.ACTIONS]
values

[107,
 108,
 106,
 104,
 117,
 110,
 98,
 121,
 75,
 76,
 74,
 72,
 85,
 78,
 66,
 89,
 60,
 62,
 46,
 13,
 35,
 191,
 225,
 193,
 97,
 24,
 64,
 67,
 90,
 227,
 99,
 195,
 228,
 100,
 68,
 101,
 69,
 229,
 27,
 70,
 102,
 230,
 59,
 86,
 105,
 73,
 233,
 234,
 4,
 92,
 96,
 58,
 236,
 237,
 109,
 77,
 239,
 111,
 79,
 15,
 112,
 44,
 240,
 80,
 113,
 241,
 81,
 114,
 18,
 82,
 210,
 242,
 103,
 71,
 83,
 115,
 42,
 34,
 91,
 36,
 61,
 43,
 40,
 94,
 41,
 33,
 243,
 120,
 84,
 65,
 20,
 116,
 212,
 95,
 244,
 88,
 245,
 246,
 118,
 87,
 38,
 47,
 119,
 247,
 122,
 43,
 45,
 32,
 39,
 34,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 36]

In [29]:
map = dict(zip(values, range(len(values))))
map

{107: 0,
 108: 1,
 106: 2,
 104: 3,
 117: 4,
 110: 5,
 98: 6,
 121: 7,
 75: 8,
 76: 9,
 74: 10,
 72: 11,
 85: 12,
 78: 13,
 66: 14,
 89: 15,
 60: 16,
 62: 17,
 46: 18,
 13: 19,
 35: 20,
 191: 21,
 225: 22,
 193: 23,
 97: 24,
 24: 25,
 64: 26,
 67: 27,
 90: 28,
 227: 29,
 99: 30,
 195: 31,
 228: 32,
 100: 33,
 68: 34,
 101: 35,
 69: 36,
 229: 37,
 27: 38,
 70: 39,
 102: 40,
 230: 41,
 59: 42,
 86: 43,
 105: 44,
 73: 45,
 233: 46,
 234: 47,
 4: 48,
 92: 49,
 96: 50,
 58: 51,
 236: 52,
 237: 53,
 109: 54,
 77: 55,
 239: 56,
 111: 57,
 79: 58,
 15: 59,
 112: 60,
 44: 61,
 240: 62,
 80: 63,
 113: 64,
 241: 65,
 81: 66,
 114: 67,
 18: 68,
 82: 69,
 210: 70,
 242: 71,
 103: 72,
 71: 73,
 83: 74,
 115: 75,
 42: 76,
 34: 109,
 91: 78,
 36: 120,
 61: 80,
 43: 105,
 40: 82,
 94: 83,
 41: 84,
 33: 85,
 243: 86,
 120: 87,
 84: 88,
 65: 89,
 20: 90,
 116: 91,
 212: 92,
 95: 93,
 244: 94,
 88: 95,
 245: 96,
 246: 97,
 118: 98,
 87: 99,
 38: 100,
 47: 101,
 119: 102,
 247: 103,
 122: 104,
 45: 106,
 3

## Dataset Configuration Options
`shuffle`: While states within a trajectory are always returned sequentially, it is possible to turn on shuffling of the *gameids*.  When true, the order of the gameids sampled is shuffled but not the order of the `seq_length` chunks returned within a single gameid.

`loop_forever`: It is possible to have the iterator loop forever instead of cycling only through the dataset once.

`gameids`: You can specify a list of gameids to return instead of iterating through the full dataset.

`subselect_sql`: And, you can select even more complicated sets of games using specific sql queries.

**NB** A `gameid` of 0 indicates that that index is padded (with 0's).

**Example 1:** Lets create a small dataset of just 4 games, and see the shuffle functionality:

In [26]:
shuffle_small_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=2,
    seq_length=6000,
    dbfilename=dbfilename,
    shuffle=True,
    loop_forever=False,
    gameids=[34,550,45],
)
for epoch in range(3):
    print(f"Epoch: {epoch}")
    for ind, mb in enumerate(shuffle_small_dataset):
        gameids = mb["gameids"][:, 0]
        print(f"  Batch {ind} first timestep gameids: {gameids}")
    print()


Epoch: 0
  Batch 0 first timestep gameids: [550  34]
  Batch 0 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 1 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 2 first timestep gameids: [550  34]
  Batch 3 first timestep gameids: [550  45]
  Batch 3 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 4 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 5 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 6 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 7 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 8 first timestep gameids: [550  45]
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
  Batch 9 first timestep gameids: [550  45]
  Batch 10 first timestep gameids: [550   0]

Epoch: 1
 

**Example 2:** We can train just on the data from a specific character, such as "mon-hum-neu-mal" by using the subselect_sql:

In [32]:
# Build the subselect sql query
subselect_sql = "SELECT gameid FROM games WHERE role=? AND race=?"
subselect_sql_args = ("Mon", "Hum")
batch_size = 10

# Build the dataset
monk_dataset = nld.TtyrecDataset(
    "taster-dataset",
    batch_size=batch_size,
    seq_length=2,
    dbfilename=dbfilename,
    subselect_sql=subselect_sql,
    subselect_sql_args=subselect_sql_args
)

# See from the error how there are fewer than 10k games despite the full dataset having 109k
print(f"Full Dataset has {nld.db.count_games('taster-dataset', conn=db_conn):,} games.")
print(f"Human Monk Subdataset Has: {len(monk_dataset._gameids)} games")

mb = next(iter(monk_dataset))

batch_idx = 0
time_idx = 0
chars = mb['tty_chars'][batch_idx, time_idx]
colors = mb['tty_colors'][batch_idx, time_idx]
cursor = mb['tty_cursor'][batch_idx, time_idx]

print(tty_render(chars, colors, cursor))

Full Dataset has 1,934 games.
Human Monk Subdataset Has: 142 games

Hello Agent, welcome to NetHack!  You are a neutral female human Monk.          
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                                                                
                                         

**Example 3**: Using a threadpool
You can also use a threadpool with the dataset which will speed it up considerably!

In [34]:
from concurrent.futures import ThreadPoolExecutor
import time


with ThreadPoolExecutor(max_workers=10) as tp:
    dataset = nld.TtyrecDataset(
        "taster-dataset",
        batch_size=100,
        seq_length=100,
        dbfilename=dbfilename,
        threadpool=tp
    )
    start = time.time()
    for i, mb in enumerate(dataset):
        if i == 10:
            break
    end = time.time()
    chars = mb['tty_chars'][batch_idx, time_idx]
    colors = mb['tty_colors'][batch_idx, time_idx]
    cursor = mb['tty_cursor'][batch_idx, time_idx]

    print(tty_render(chars, colors, cursor))
# NB this might be v slow on free Colab, try on laptop or server.
print(f"Loaded 100,000 frames in {end-start:.2f}s")


                                                                                
                                                                                
                              -----------                                       
                              |..Z......|              ----------------         
                              |..........#           ##...:...........|         
                              |.........|#           # |._............|         
                              ---------|-#           # |..............|         
        ----------------            #### #           # -.--------------         
        |.......{..<...-###    ######    ##          #  #####                   
        |..............|  ##   #          #        ###      #                   
        |...............@#d#####          #      ###        #######             
        ----------------    #-|-----      #-----##                #             
                           

**Example 4:** Getting Metadata

In [ ]:
dataset = nld.TtyrecDataset('taster-dataset', dbfilename=dbfilename)
mb = next(iter(dataset))
gameid = mb["gameids"][0][0]

chars = mb['tty_chars'][0, 0]
colors = mb['tty_colors'][0, 0]
cursor = mb['tty_cursor'][0, 0]

print(tty_render(chars, colors, cursor))

dict(dataset.get_meta(gameid))


Hello Agent, welcome to NetHack!  You are a neutral male gnomish Archeologist.  
                                                                                
                                                                                
                                                                                
                               --------                                         
                               |....$.|                                         
                               |..!{..|                                         
                               |......|                                         
                               |...@f(|                                         
                               |......|                                         
                               |.....)|                                         
                               --.-----                                         
                           

{'gameid': 816,
 'version': '3.6.6',
 'points': 7515,
 'deathdnum': 0,
 'deathlev': 5,
 'maxlvl': 5,
 'hp': 0,
 'maxhp': 49,
 'deaths': 1,
 'deathdate': 20220518,
 'birthdate': 20220518,
 'uid': 1185200751,
 'role': 'Arc',
 'race': 'Gno',
 'gender': 'Mal',
 'align': 'Neu',
 'name': 'Agent',
 'death': 'killed by a white unicorn',
 'conduct': '0xfc0',
 'turns': 22750,
 'achieve': '0x0',
 'realtime': 142,
 'starttime': 1652882603,
 'endtime': 1652882745,
 'gender0': 'Mal',
 'align0': 'Neu',
 'flags': '0x4'}

**Example 5** Generating and loading a custom dataset.

In [ ]:
import gym
import nle
import nle.dataset as nld
from datetime import datetime

def generate_rollouts(env):
    obs = env.reset()
    episodes = 0
    while episodes < 10:
        obs, reward, done, info = env.step(env.action_space.sample())
        if done:
            env.reset()
            episodes += 1

# 1. Create some envs, with a savedir directory 'path/to/save/X'
envA = gym.make("NetHackChallenge-v0", savedir="path/to/save/A", save_ttyrec_every=2)
envB = gym.make("NetHackScore-v0", character="Mon-Hum-Neu-Mal", savedir="path/to/save/B", save_ttyrec_every=1)

# 2. Generate rollouts
generate_rollouts(envA)
generate_rollouts(envB)

# 3. Add to directory, with given unique dataset name
name = f"dataset_{datetime.now().time()}"
if not nld.db.exists():
    nld.db.create()
nld.add_nledata_directory("path/to/save", name)

# 4. Use and enjoy!
dataset = nld.TtyrecDataset(name)
print(f"Dataset has {len(dataset._gameids)} entries!")



Adding dataset 'dataset_15:38:53.943302' ('path/to/save') to 'ttyrecs.db' 
Updated 'ttyrecs.db' in 0.00 sec. Size: 0.65 MB, Games: 15
Dataset has 15 entries!


**Example 6:** Use doctstrings - don't forget a lot of the classes and methods have docstrings. Have fun!

In [ ]:
help(nld.TtyrecDataset)

Help on class TtyrecDataset in module nle.dataset.dataset:

class TtyrecDataset(builtins.object)
 |  TtyrecDataset(dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |  
 |  Dataset object to allow iteration through the ttyrecs found in our ttyrec
 |  database.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, dataset_name, batch_size=128, seq_length=32, rows=24, cols=80, dbfilename='ttyrecs.db', threadpool=None, gameids=None, shuffle=True, loop_forever=False, subselect_sql=None, subselect_sql_args=None)
 |      An iterable dataset to load minibatches of NetHack games from compressed
 |      ttyrec*.bz2 files into numpy arrays. (shape: [batch_size, seq_length, ...])
 |      
 |      This class makes use of a sqlite3 database at `dbfilename` to find the
 |      metadata and the location of files in a dataset. It then uses these to
 |   

In [41]:
# Test the new keypress to action index functions
import importlib
import utils.action_utils
importlib.reload(utils.action_utils)
from utils.action_utils import keypress_to_action_index, batch_keypress_to_action_index

print("=== Testing Keypress to Action Index Conversion ===")

# Test single keypress conversion
test_keypresses = [107, 106, 108, 104, 46, 121, 117, 110, 98]  # k, j, l, h, ., y, u, n, b
test_chars = ['k', 'j', 'l', 'h', '.', 'y', 'u', 'n', 'b']

print("Single keypress conversions:")
for keypress, char in zip(test_keypresses, test_chars):
    try:
        action_index = keypress_to_action_index(env, keypress)
        action = env.actions[action_index]
        print(f"  Keypress {keypress} ('{char}') -> Action index {action_index} -> {action.name}")
    except Exception as e:
        print(f"  Keypress {keypress} ('{char}') -> Error: {e}")

print("\nBatch keypress conversion:")
import torch
keypresses_tensor = torch.tensor(test_keypresses)
action_indices = batch_keypress_to_action_index(env, keypresses_tensor)
print(f"  Input keypresses: {test_keypresses}")
print(f"  Output action indices: {action_indices.tolist()}")

# Compare with the manual embedding from the tutorial
manual_embed_actions = torch.zeros((256, 1))
for i, a in enumerate(env.actions):
    manual_embed_actions[a.value][0] = i
manual_embed_layer = torch.nn.Embedding.from_pretrained(manual_embed_actions)
manual_result = manual_embed_layer(keypresses_tensor).squeeze(-1).long()
print(f"  Manual embedding result: {manual_result.tolist()}")
print(f"  Results match: {torch.equal(action_indices, manual_result)}")

# Test with the minibatch keypresses
print(f"\nTesting with minibatch keypresses:")
sample_keypresses = minibatch['keypresses'][0, :10]  # First 10 keypresses from first batch
print(f"  Sample keypresses: {sample_keypresses.tolist()}")
sample_actions = batch_keypress_to_action_index(env, sample_keypresses)
print(f"  Converted to actions: {sample_actions.tolist()}")

# Show what these actions represent
print("  Action details:")
for i, (keypress, action_idx) in enumerate(zip(sample_keypresses, sample_actions)):
    if i < 5:  # Show first 5 only
        action = env.actions[action_idx.item()]
        char = chr(keypress.item()) if 32 <= keypress.item() <= 126 else f"\\x{keypress.item():02x}"
        print(f"    {keypress.item()} ('{char}') -> {action.name}")

=== Testing Keypress to Action Index Conversion ===
Single keypress conversions:
  Keypress 107 ('k') -> Action index 0 -> N
  Keypress 106 ('j') -> Action index 2 -> S
  Keypress 108 ('l') -> Action index 1 -> E
  Keypress 104 ('h') -> Action index 3 -> W
  Keypress 46 ('.') -> Action index 18 -> WAIT
  Keypress 121 ('y') -> Action index 7 -> NW
  Keypress 117 ('u') -> Action index 4 -> NE
  Keypress 110 ('n') -> Action index 5 -> SE
  Keypress 98 ('b') -> Action index 6 -> SW

Batch keypress conversion:
  Input keypresses: [107, 106, 108, 104, 46, 121, 117, 110, 98]
  Output action indices: [0, 2, 1, 3, 18, 7, 4, 5, 6]
  Manual embedding result: [0, 2, 1, 3, 18, 7, 4, 5, 6]
  Results match: True

Testing with minibatch keypresses:
  Sample keypresses: [27, 27, 24, 32, 32, 229, 32, 32, 64, 92]
  Converted to actions: [38, 38, 25, 107, 107, 37, 107, 107, 26, 49]
  Action details:
    27 ('\x1b') -> ESC
    27 ('\x1b') -> ESC
    24 ('\x18') -> ATTRIBUTES
    32 (' ') -> SPACE
    32 ('

In [34]:
# Test the improved batch_keypress_static_map with n-dimensional tensors
import importlib
import utils.action_utils
importlib.reload(utils.action_utils)
from utils.action_utils import batch_keypress_static_map, batch_keypress_to_action_index

print("=== Testing batch_keypress_static_map with N-Dimensional Tensors ===")

import torch

# Test 1: 1D tensor
print("1. Testing 1D tensor:")
keypresses_1d = torch.tensor([107, 106, 108, 104])  # k, j, l, h
result_1d = batch_keypress_static_map(keypresses_1d)
print(f"   Input shape: {keypresses_1d.shape}")
print(f"   Output shape: {result_1d.shape}")
print(f"   Input: {keypresses_1d.tolist()}")
print(f"   Output: {result_1d.tolist()}")

# Test 2: 2D tensor (batch_size=2, seq_length=3)
print("\n2. Testing 2D tensor:")
keypresses_2d = torch.tensor([[107, 106, 108], [104, 46, 121]])  # [k,j,l], [h,.,y]
result_2d = batch_keypress_static_map(keypresses_2d)
print(f"   Input shape: {keypresses_2d.shape}")
print(f"   Output shape: {result_2d.shape}")
print(f"   Input: {keypresses_2d.tolist()}")
print(f"   Output: {result_2d.tolist()}")

# Test 3: 3D tensor (batch_size=2, seq_length=2, features=2)
print("\n3. Testing 3D tensor:")
keypresses_3d = torch.tensor([[[107, 106], [108, 104]], [[46, 121], [117, 110]]])  # [[k,j],[l,h]], [[.,y],[u,n]]
result_3d = batch_keypress_static_map(keypresses_3d)
print(f"   Input shape: {keypresses_3d.shape}")
print(f"   Output shape: {result_3d.shape}")
print(f"   Input: {keypresses_3d.tolist()}")
print(f"   Output: {result_3d.tolist()}")

# Test 4: List input
print("\n4. Testing list input:")
keypresses_list = [107, 106, 108, 104]
result_list = batch_keypress_static_map(keypresses_list)
print(f"   Input: {keypresses_list}")
print(f"   Output shape: {result_list.shape}")
print(f"   Output: {result_list.tolist()}")

# Test 5: Real minibatch data (2D)
print("\n5. Testing with real minibatch data:")
sample_keypresses = minibatch['keypresses'][:2, :5]  # First 2 batches, first 5 timesteps
result_minibatch = batch_keypress_static_map(sample_keypresses)
print(f"   Input shape: {sample_keypresses.shape}")
print(f"   Output shape: {result_minibatch.shape}")
print(f"   Input: {sample_keypresses.tolist()}")
print(f"   Output: {result_minibatch.tolist()}")

# Verify it produces same results as the original env-based function
print("\n6. Verification against original function:")
test_keypresses = torch.tensor([107, 106, 108, 104])
static_result = batch_keypress_static_map(test_keypresses)
env_result = batch_keypress_to_action_index(env, test_keypresses)
print(f"   Static map result: {static_result.tolist()}")
print(f"   Env-based result: {env_result.tolist()}")
print(f"   Results match: {torch.equal(static_result, env_result)}")

print("\n✅ All tests completed successfully!")

=== Testing batch_keypress_static_map with N-Dimensional Tensors ===
1. Testing 1D tensor:
   Input shape: torch.Size([4])
   Output shape: torch.Size([4])
   Input: [107, 106, 108, 104]
   Output: [0, 2, 1, 3]

2. Testing 2D tensor:
   Input shape: torch.Size([2, 3])
   Output shape: torch.Size([2, 3])
   Input: [[107, 106, 108], [104, 46, 121]]
   Output: [[0, 2, 1], [3, 18, 7]]

3. Testing 3D tensor:
   Input shape: torch.Size([2, 2, 2])
   Output shape: torch.Size([2, 2, 2])
   Input: [[[107, 106], [108, 104]], [[46, 121], [117, 110]]]
   Output: [[[0, 2], [1, 3]], [[18, 7], [4, 5]]]

4. Testing list input:
   Input: [107, 106, 108, 104]
   Output shape: torch.Size([4])
   Output: [0, 2, 1, 3]

5. Testing with real minibatch data:
   Input shape: (2, 5)
   Output shape: torch.Size([2, 5])
   Input: [[32, 27, 27, 24, 32], [32, 27, 27, 24, 32]]
   Output: [[107, 38, 38, 25, 107], [107, 38, 38, 25, 107]]

6. Verification against original function:
   Static map result: [0, 2, 1, 3]
  

In [35]:
# Test the optimized KEYPRESS_INDEX_MAPPING hash table
import importlib
import utils.action_utils
importlib.reload(utils.action_utils)
from utils.action_utils import (
    get_keypress_mapping, 
    is_valid_keypress, 
    keypress_static_map, 
    batch_keypress_static_map,
    KEYPRESS_INDEX_MAPPING
)

print("=== Testing Optimized Hash Table KEYPRESS_INDEX_MAPPING ===")

# Test 1: Direct access to the hash table
print("1. Testing direct access to KEYPRESS_INDEX_MAPPING:")
print(f"   Type: {type(KEYPRESS_INDEX_MAPPING)}")
print(f"   Size: {len(KEYPRESS_INDEX_MAPPING)} mappings")
print(f"   Sample mappings:")
sample_keys = [107, 106, 108, 104, 46]  # k, j, l, h, .
for key in sample_keys:
    if key in KEYPRESS_INDEX_MAPPING:
        char = chr(key) if 32 <= key <= 126 else f"\\x{key:02x}"
        print(f"     {key} ('{char}') -> action index {KEYPRESS_INDEX_MAPPING[key]}")

# Test 2: get_keypress_mapping() function
print("\n2. Testing get_keypress_mapping() function:")
mapping_copy = get_keypress_mapping()
print(f"   Retrieved mapping size: {len(mapping_copy)}")
print(f"   Is copy (not same object): {mapping_copy is not KEYPRESS_INDEX_MAPPING}")
print(f"   Contents match: {mapping_copy == KEYPRESS_INDEX_MAPPING}")

# Test 3: is_valid_keypress() function
print("\n3. Testing is_valid_keypress() function:")
test_keypresses = [107, 106, 108, 104, 46, 999, 256, -1]
test_chars = ['k', 'j', 'l', 'h', '.', '999', '256', '-1']
for keypress, char in zip(test_keypresses, test_chars):
    valid = is_valid_keypress(keypress)
    print(f"   is_valid_keypress({keypress}) ('{char}'): {valid}")

# Test 4: keypress_static_map() with improved error handling
print("\n4. Testing keypress_static_map() with hash table:")
import time

# Test valid keypresses
valid_keypresses = [107, 106, 108, 104, 46]
start_time = time.time()
for _ in range(1000):  # Performance test
    for keypress in valid_keypresses:
        result = keypress_static_map(keypress)
end_time = time.time()

print(f"   Performance test (5000 lookups): {(end_time - start_time)*1000:.2f}ms")
print(f"   Valid keypresses:")
for keypress in valid_keypresses:
    char = chr(keypress) if 32 <= keypress <= 126 else f"\\x{keypress:02x}"
    result = keypress_static_map(keypress)
    print(f"     {keypress} ('{char}') -> {result}")

# Test invalid keypress
print(f"   Invalid keypress test:")
try:
    result = keypress_static_map(999)
    print(f"     keypress_static_map(999) -> {result}")
except KeyError as e:
    print(f"     KeyError (expected): {e}")

# Test 5: batch_keypress_static_map() with optimized lookup
print("\n5. Testing batch_keypress_static_map() with optimized hash table:")
import torch

# Performance comparison test
test_data = torch.tensor([107, 106, 108, 104, 46] * 1000)  # 5000 elements

start_time = time.time()
result_optimized = batch_keypress_static_map(test_data)
end_time = time.time()

print(f"   Batch processing performance (5000 elements): {(end_time - start_time)*1000:.2f}ms")
print(f"   Input shape: {test_data.shape}")
print(f"   Output shape: {result_optimized.shape}")
print(f"   Sample results: {result_optimized[:10].tolist()}")

# Test 6: Verify consistency between functions
print("\n6. Testing consistency between different functions:")
test_keypresses = torch.tensor([107, 106, 108, 104, 46])

# Single function results
single_results = [keypress_static_map(k.item()) for k in test_keypresses]

# Batch function result
batch_results = batch_keypress_static_map(test_keypresses).tolist()

# Hash table direct lookup
direct_results = [KEYPRESS_INDEX_MAPPING[k.item()] for k in test_keypresses]

print(f"   Single function results: {single_results}")
print(f"   Batch function results:  {batch_results}")
print(f"   Direct hash table:       {direct_results}")
print(f"   All methods consistent:  {single_results == batch_results == direct_results}")

print("\n✅ Hash table optimization tests completed successfully!")

=== Testing Optimized Hash Table KEYPRESS_INDEX_MAPPING ===
1. Testing direct access to KEYPRESS_INDEX_MAPPING:
   Type: <class 'dict'>
   Size: 118 mappings
   Sample mappings:
     107 ('k') -> action index 0
     106 ('j') -> action index 2
     108 ('l') -> action index 1
     104 ('h') -> action index 3
     46 ('.') -> action index 18

2. Testing get_keypress_mapping() function:
   Retrieved mapping size: 118
   Is copy (not same object): True
   Contents match: True

3. Testing is_valid_keypress() function:
   is_valid_keypress(107) ('k'): True
   is_valid_keypress(106) ('j'): True
   is_valid_keypress(108) ('l'): True
   is_valid_keypress(104) ('h'): True
   is_valid_keypress(46) ('.'): True
   is_valid_keypress(999) ('999'): False
   is_valid_keypress(256) ('256'): False
   is_valid_keypress(-1) ('-1'): False

4. Testing keypress_static_map() with hash table:
   Performance test (5000 lookups): 0.42ms
   Valid keypresses:
     107 ('k') -> 0
     106 ('j') -> 2
     108 ('l') 

In [ ]:
# Performance test to demonstrate O(1) lookup time of Python dict (hash table)
import time
import random
from utils.action_utils import KEYPRESS_INDEX_MAPPING

print("=== Python Dict (Hash Table) Performance Analysis ===")

# Test 1: Confirm Python dict is a hash table
print("1. Python dict implementation verification:")
print(f"   KEYPRESS_INDEX_MAPPING type: {type(KEYPRESS_INDEX_MAPPING)}")
print(f"   Size: {len(KEYPRESS_INDEX_MAPPING)} entries")

# Create test dictionaries of different sizes to show O(1) behavior
def create_test_dict(size):
    """Create a test dictionary of given size"""
    return {i: f"value_{i}" for i in range(size)}

# Test different dictionary sizes
sizes = [100, 1000, 10000, 100000]
lookup_times = []

print("\n2. Performance test with different dictionary sizes:")
print("   Size     | Avg Lookup Time | Time per 1000 lookups")
print("   ---------|-----------------|---------------------")

for size in sizes:
    test_dict = create_test_dict(size)
    keys_to_test = random.sample(list(test_dict.keys()), min(1000, size))
    
    # Time the lookups
    start_time = time.time()
    for _ in range(1000):  # Do 1000 iterations for better measurement
        for key in keys_to_test[:100]:  # Test 100 keys each iteration
            _ = test_dict[key]  # Dictionary lookup
    end_time = time.time()
    
    total_time = (end_time - start_time) * 1000  # Convert to milliseconds
    avg_time_per_lookup = total_time / (1000 * 100)  # Time per single lookup
    time_per_1000 = total_time / 1000  # Time per 1000 lookups
    
    lookup_times.append((size, avg_time_per_lookup, time_per_1000))
    print(f"   {size:8d} | {avg_time_per_lookup:13.6f}ms | {time_per_1000:17.3f}ms")

# Test 3: Compare with list (O(n)) lookup
print("\n3. Comparison: Dict O(1) vs List O(n) lookup:")

# Create equivalent list and dict
test_size = 1000
test_dict = create_test_dict(test_size)
test_list = [(i, f"value_{i}") for i in range(test_size)]
test_keys = random.sample(list(range(test_size)), 100)

# Time dict lookup (O(1))
start_time = time.time()
for _ in range(100):
    for key in test_keys:
        _ = test_dict[key]
dict_time = (time.time() - start_time) * 1000

# Time list lookup (O(n))
start_time = time.time()
for _ in range(100):
    for key in test_keys:
        # Linear search in list
        for k, v in test_list:
            if k == key:
                _ = v
                break
list_time = (time.time() - start_time) * 1000

print(f"   Dict lookup (O(1)): {dict_time:.3f}ms")
print(f"   List lookup (O(n)): {list_time:.3f}ms")
print(f"   Dict is {list_time/dict_time:.1f}x faster")

# Test 4: NetHack keypress mapping performance
print("\n4. NetHack keypress mapping performance:")
test_keypresses = [107, 106, 108, 104, 46] * 200  # 1000 lookups

start_time = time.time()
for keypress in test_keypresses:
    _ = KEYPRESS_INDEX_MAPPING[keypress]  # Hash table lookup
end_time = time.time()

nethack_lookup_time = (end_time - start_time) * 1000
print(f"   1000 NetHack keypress lookups: {nethack_lookup_time:.3f}ms")
print(f"   Average per lookup: {nethack_lookup_time/1000:.6f}ms")

# Analysis
print("\n5. Analysis:")
print("   ✅ Python dict IS a hash table implementation")
print("   ✅ Lookup time remains roughly constant as size increases (O(1))")
print("   ✅ Much faster than linear search (O(n))")
print("   ✅ Our NetHack mapping uses efficient O(1) hash table lookups")

# Show the scaling behavior
print("\n6. Scaling behavior (should be roughly constant for O(1)):")
if len(lookup_times) > 1:
    base_time = lookup_times[0][1]
    for size, avg_time, _ in lookup_times:
        ratio = avg_time / base_time
        print(f"   Size {size:6d}: {ratio:.2f}x relative to size {sizes[0]}")
    
    print(f"\n   📊 If O(log n): ratios should grow like log(size)")
    print(f"   📊 If O(1): ratios should stay close to 1.0")
    print(f"   📊 Our ratios stay close to 1.0 → Confirming O(1) behavior")